In [1]:
raw_dat_dir = '/home/grace/Documents/python/ttc_inhalation/data/raw/'
processed_dat_dir = '/home/grace/Documents/python/ttc_inhalation/data/processed/'
interim_dat_dir = '/home/grace/Documents/python/ttc_inhalation/data/interim/'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
import os
import glob

### Import chemical properties

In [4]:
chem_prop = pd.read_csv(processed_dat_dir+'ToxVal_v7_chem_prop.csv')

In [5]:
chem_prop.shape

(4703, 62)

In [6]:
toxval = pd.read_csv(processed_dat_dir+'ToxVal_wstudylength.csv')

/home/grace/anaconda3/envs/clone_newchem/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (26,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#toxval.columns.tolist()
#toxval.toxval_units.unique()

In [8]:
#toxval.exposure_route.unique()

In [9]:
toxval_chemprop = pd.merge(toxval, chem_prop, on = 'DTXSID', how = 'left')

In [10]:
toxval_chemprop.shape

(675694, 114)

### ppm to mgm3 unit conversions: Basic equation for conversions: mg/m3 = ppm* Mol Weight/24.45

In [11]:
mgm3_units = []
for x, y, z, a in list(zip(toxval_chemprop['exposure_route'], toxval_chemprop['toxval_units'], toxval_chemprop['MolWeight'], toxval_chemprop['toxval_numeric'])):
    if (x == 'inhalation' and y == 'ppm') or (x == 'inhalation' and y == 'mg/L'):
        xx = (a*z)/24.45
        mgm3_units.append(xx)
    elif (x == "inhalation" and y == "g/m3"):
        mgm3_units.append(a * 1000)
    elif (x == "inhalation" and y == "ug/m3"):
        mgm3_units.append(a / 1000)
    elif (x == "inhalation" and y == "mg/m") or (x == "inhalation" and y == "mg/m3"):
        mgm3_units.append(a)
    else:
        mgm3_units.append(a)
    

In [12]:
len(mgm3_units)

675694

### mg/m3 to ppm conversion: Basic equation used: ppm = (24.45 * mg/m3) / Mol Weight

In [13]:
ppm_units = []
for x, y, z, a in list(zip(toxval_chemprop['exposure_route'], toxval_chemprop['toxval_units'], toxval_chemprop['MolWeight'], toxval_chemprop['toxval_numeric'])):
    if (x == 'inhalation' and y == 'mg/m') or (x == 'inhalation' and y == 'mg/m3'):
        xx = (a*24.45)/z
        ppm_units.append(xx)
    elif (x == "inhalation" and y == "g/m3"):
        xy = (24.45 * (a * 1000)) / z
        ppm_units.append(xy)
    elif (x == "inhalation" and y == "ug/m3"):
        bb = (24.45 * (a/1000)) / z
        ppm_units.append(bb)
    elif (x == "inhalation" and y == "ppm") or (x == "inhalation" and y == "mg/L"):
        ppm_units.append(a)
    else:
        ppm_units.append(a)
    

In [14]:
len(ppm_units)

675694

In [15]:
toxval_chemprop['toxval_ppm'] = ppm_units
toxval_chemprop['toxval_mgm3'] = mgm3_units

In [16]:
toolbox = pd.read_csv(processed_dat_dir+'Toolbox_out.csv')

In [17]:
toolbox.head()

,CAS,Chemical name(s),SMILES,Molecular Formula,Predefined substance type,Additional Ids,Composition,CAS Smiles relation,Eye irritation/corrosion Exclusion rules by BfR,Eye irritation/corrosion Inclusion rules by BfR,Respiratory sensitisation,Skin irritation/corrosion Exclusion rules by BfR,Skin irritation/corrosion Inclusion rules by BfR
0,No CAS number,38480;Ethylbenzene;DTXSID3020596,CCc1ccccc1,C8H10,Mono constituent,NaN,NaN,Not applicable,Undefined,Inclusion rules not met,No alert found,Group C Surface Tension > 62 mN/m;Undefined,Inclusion rules not met
1,No CAS number,vinylbenzene;DTXSID2021284,C=Cc1ccccc1,C8H8,Mono constituent,NaN,NaN,Not applicable,Undefined,Inclusion rules not met,Pro-SN2;Pro-SN2 >> Pro-ring opening SN2;Pro-SN...,Undefined,Inclusion rules not met
2,No CAS number,DTXSID5020865,Nc1ccc(Cc2ccc(N)c(Cl)c2)cc1Cl,C13H12Cl2N2,Mono constituent,NaN,NaN,Not applicable,Group All Melting Point > 200 C;Group CNHal Aq...,Inclusion rules not met,No alert found,Group All Melting Point > 200 C;Group CNHal Aq...,Aromatic amines
3,No CAS number,DTXSID7025180,O=C=Nc1ccc(Cc2ccc(cc2)N=C=O)cc1,C15H10N2O2,Mono constituent,NaN,NaN,Not applicable,Group CN Aqueous Solubility < 0.1 g/L;Group CN...,Inclusion rules not met,Acylation;Acylation >> Isocyanates and related...,Group CN Aqueous Solubility < 0.1 g/L;Group CN...,Inclusion rules not met
4,No CAS number,DTXSID6022422,Nc1ccc(Cc2ccc(N)cc2)cc1,C13H14N2,Mono constituent,NaN,NaN,Not applicable,Undefined,Inclusion rules not met,No alert found,Group CN Vapour Pressure < 0.001 Pa;Undefined,Aromatic amines


In [18]:
import re
p = re.compile(r'DTXSID\d{1,}')

In [19]:
dtxsid = [m.group(0) for l in toolbox['Chemical name(s)'] for m in [p.search(l)] if m]

In [20]:
len(dtxsid)

4702

In [21]:
toolbox = toolbox[toolbox['Chemical name(s)'].str.contains('DTXSID')]

In [22]:
toolbox['DTXSID'] = dtxsid

In [23]:
toolbox.drop(['CAS', 'Chemical name(s)', 'Additional Ids', 'Composition', 'CAS Smiles relation'], axis = 1, inplace = True)


In [24]:
toolbox = toolbox[['DTXSID','SMILES','Eye irritation/corrosion Exclusion rules by BfR',
       'Eye irritation/corrosion Inclusion rules by BfR',
       'Respiratory sensitisation',
       'Skin irritation/corrosion Exclusion rules by BfR',
       'Skin irritation/corrosion Inclusion rules by BfR' ]]

In [25]:
toolbox.columns = ['DTXSID','SMILES','BfR_eye_exclusion', 'BfR_eye_inclusion', 'resp_sens', 'BfR_skin_exclusion', 'BfR_skin_inclusion']

In [26]:
toolbox.head()

,DTXSID,SMILES,BfR_eye_exclusion,BfR_eye_inclusion,resp_sens,BfR_skin_exclusion,BfR_skin_inclusion
0,DTXSID3020596,CCc1ccccc1,Undefined,Inclusion rules not met,No alert found,Group C Surface Tension > 62 mN/m;Undefined,Inclusion rules not met
1,DTXSID2021284,C=Cc1ccccc1,Undefined,Inclusion rules not met,Pro-SN2;Pro-SN2 >> Pro-ring opening SN2;Pro-SN...,Undefined,Inclusion rules not met
2,DTXSID5020865,Nc1ccc(Cc2ccc(N)c(Cl)c2)cc1Cl,Group All Melting Point > 200 C;Group CNHal Aq...,Inclusion rules not met,No alert found,Group All Melting Point > 200 C;Group CNHal Aq...,Aromatic amines
3,DTXSID7025180,O=C=Nc1ccc(Cc2ccc(cc2)N=C=O)cc1,Group CN Aqueous Solubility < 0.1 g/L;Group CN...,Inclusion rules not met,Acylation;Acylation >> Isocyanates and related...,Group CN Aqueous Solubility < 0.1 g/L;Group CN...,Inclusion rules not met
4,DTXSID6022422,Nc1ccc(Cc2ccc(N)cc2)cc1,Undefined,Inclusion rules not met,No alert found,Group CN Vapour Pressure < 0.001 Pa;Undefined,Aromatic amines


In [27]:
toolbox['BfR_skin_inclusion'] = [e.replace(';','|') for e in toolbox['BfR_skin_inclusion'] ]


In [28]:
toolbox['BfR_eye_inclusion'] = [e.replace(';','|') for e in toolbox['BfR_eye_inclusion'] ]

In [29]:
toxval_chemprop.shape

(675694, 116)

In [30]:
toxval_chemprop[toxval_chemprop['DTXSID'] == 'DTXSID3020596'].shape

(389, 116)

In [31]:
toxval_prop = pd.merge(toxval_chemprop, toolbox, on = 'DTXSID', how = 'left')

In [32]:
toxval_prop.to_csv(processed_dat_dir+'ToxVal_wpred_properties.csv')

In [33]:
toxval_prop.shape

(675694, 122)

In [34]:
toxval_prop.head()

,Unnamed: 0_x,casrn,name,DTXSID,dsstox_compound_id,qsar_ready_smiles,chemical_id_original,quality_id,source,subsource,...,AD_index_AOH,Conf_index_AOH,toxval_ppm,toxval_mgm3,SMILES,BfR_eye_exclusion,BfR_eye_inclusion,resp_sens,BfR_skin_exclusion,BfR_skin_inclusion
0,0,100-41-4,Ethylbenzene,DTXSID3020596,DTXCID10596,CCC1=CC=CC=C1,147,-1,ATSDR,CDC,...,0.911374,0.817831,2.00,8.677157,CCc1ccccc1,Undefined,Inclusion rules not met,No alert found,Group C Surface Tension > 62 mN/m;Undefined,Inclusion rules not met
1,1,100-41-4,Ethylbenzene,DTXSID3020596,DTXCID10596,CCC1=CC=CC=C1,147,-1,ATSDR,CDC,...,0.911374,0.817831,18.00,78.094417,CCc1ccccc1,Undefined,Inclusion rules not met,No alert found,Group C Surface Tension > 62 mN/m;Undefined,Inclusion rules not met
2,2,100-41-4,Ethylbenzene,DTXSID3020596,DTXCID10596,CCC1=CC=CC=C1,147,-1,ATSDR,CDC,...,0.911374,0.817831,5.00,21.692894,CCc1ccccc1,Undefined,Inclusion rules not met,No alert found,Group C Surface Tension > 62 mN/m;Undefined,Inclusion rules not met
3,3,100-41-4,Ethylbenzene,DTXSID3020596,DTXCID10596,CCC1=CC=CC=C1,147,-1,ATSDR,CDC,...,0.911374,0.817831,12.00,12.000000,CCc1ccccc1,Undefined,Inclusion rules not met,No alert found,Group C Surface Tension > 62 mN/m;Undefined,Inclusion rules not met
4,4,100-41-4,Ethylbenzene,DTXSID3020596,DTXCID10596,CCC1=CC=CC=C1,147,-1,ATSDR,CDC,...,0.911374,0.817831,0.06,0.260315,CCc1ccccc1,Undefined,Inclusion rules not met,No alert found,Group C Surface Tension > 62 mN/m;Undefined,Inclusion rules not met
